In [1]:
import numpy as np
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import tensorflow as tf
from IPython.display import display, Image
import shutil
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os

In [13]:
with open('train_ids.txt', 'r') as file:
    # Read all lines into a list
    lines = file.readlines()
    train_ids = []

    # Print each line
    for line in lines:
        train_ids.append(line.strip() + '.jpg')

In [3]:
import json

# Specify the path to your JSON file
json_file_path = '/Users/raghavraahul/Downloads/Machine Learning under a Modern Optimization Lens/ML Project/MMHS150K_GT.json'

# Open the JSON file for reading
with open(json_file_path, 'r') as json_file:
    # Load the JSON data
    data = json.load(json_file)

In [4]:
df = pd.DataFrame.from_dict(data, orient='index')
df = df.reset_index()
newColumns = {"index": "id"}
df = df.rename(columns=newColumns)

In [5]:
#0 for nothate, #1 for hate
def labels_to_binary(labels):
    if sum(labels) <= 1:
        return 0
    else:
        return 1

In [6]:
df['id_with_extension'] = df['id'] + '.jpg'
df['label'] = df['labels'].apply(labels_to_binary)

In [10]:
hate_df = df[df['label'] == 1]

In [11]:
nothate_df = df[df['label'] == 0]

In [95]:
"""hate_train_ids = []
count = 0
for id_ in hate_df['id_with_extension']:
    if id_ in train_ids:
        hate_train_ids.append(id_)
        count += 1
        if count == 10000:
            break
""";
hate_train_ids = []
count = 0
for id_ in hate_df['id_with_extension'].tolist():
    if id_ in text_train_embedding['id_with_extension'].tolist():
        hate_train_ids.append(id_)
        count += 1
        if count == 10000:
            break

In [18]:
hate_df['id_with_extension'][0]

'1114679353714016256.jpg'

In [19]:
train_ids[0]

'1057393737372966912.jpg'

In [91]:
#text_train_embedding['id_with_extension'] = text_train_embedding['id'] + '.jpg'
nothate_df['id_with_extension'].tolist()

['1108927368075374593.jpg',
 '1114558534635618305.jpg',
 '1106978219654303744.jpg',
 '1114588617693966336.jpg',
 '1116702448016556035.jpg',
 '1114666757497733122.jpg',
 '1109613175861899264.jpg',
 '1050477688194969600.jpg',
 '1113530722789228549.jpg',
 '1107795783066312704.jpg',
 '1113555815829471238.jpg',
 '1106580019936182273.jpg',
 '1108093145122856961.jpg',
 '1055176608624058368.jpg',
 '1044596079042408453.jpg',
 '1113145568954601472.jpg',
 '1105174167488548865.jpg',
 '1109653281435860992.jpg',
 '1107661782229884933.jpg',
 '1107260404810563584.jpg',
 '1116532229960761354.jpg',
 '1117530064130846722.jpg',
 '1109210622502285313.jpg',
 '1115275629476372480.jpg',
 '1114250985956835339.jpg',
 '1109641012090204160.jpg',
 '1056821741622910977.jpg',
 '1107619527054016512.jpg',
 '1115029160743849984.jpg',
 '1113418430328209421.jpg',
 '1105633264591687681.jpg',
 '1116683270798958592.jpg',
 '1105326061963919360.jpg',
 '1113461398238060550.jpg',
 '1110349746495983623.jpg',
 '110912802381776076

In [93]:
nothate_train_ids = []
count = 0
for id_ in nothate_df['id_with_extension'].tolist():
    if id_ in text_train_embedding['id_with_extension'].tolist():
        nothate_train_ids.append(id_)
        count += 1
        if count == 10000:
            break

In [96]:
len(nothate_train_ids)

10000

In [23]:
def copy_files(source_path, destination_path, titles):
    data_directory = source_path

    for title in titles:
        source_file = os.path.join(data_directory, title)
        destination_file = os.path.join(destination_path, title)
        shutil.copy(source_file, destination_file)

In [24]:
source_path = "/Users/raghavraahul/Downloads/Machine Learning under a Modern Optimization Lens/ML Project/img_resized/"

In [25]:
destination_path1 = "/Users/raghavraahul/Downloads/Machine Learning under a Modern Optimization Lens/ML Project/train20000/hate"
destination_path2 = "/Users/raghavraahul/Downloads/Machine Learning under a Modern Optimization Lens/ML Project/train20000/notHate"

In [27]:
len(hate_train_ids)

10000

In [97]:
copy_files(source_path, destination_path1, hate_train_ids)

In [98]:
copy_files(source_path, destination_path2, nothate_train_ids)

In [99]:
import os
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input
import cv2
import pandas as pd

# Set the path to your dataset
dataset_path = '/Users/raghavraahul/Downloads/Machine Learning under a Modern Optimization Lens/ML Project'

# Define the image size based on the pre-trained model's requirements
img_size = (224, 224)

# Define the batch size
batch_size = 32

def preprocess_with_denoising(image):
    # Gaussian Blur for noise reduction
    denoised_image = cv2.GaussianBlur(image, (5, 5), 0)
    
    # Original preprocess_input function
    preprocessed_image = preprocess_input(denoised_image)

    return preprocessed_image

In [100]:
# ImageDataGenerator with noise reduction in preprocessing
datagen = ImageDataGenerator(
    preprocessing_function=preprocess_with_denoising,
    rescale=1./255,  # Rescale pixel values to [0, 1]
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Create a MobileNetV2 model (assuming weights are set to 'imagenet')
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Create a data generator for your dataset
data_generator = datagen.flow_from_directory(
    os.path.join(dataset_path, 'train20000'),
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',  # Use 'categorical' for one-hot encoded labels
    shuffle=False  # Set to False if you want to keep the order of predictions
)

Found 20000 images belonging to 2 classes.


In [101]:
# Generate embeddings for your images
image_embeddings = base_model.predict(data_generator, verbose=1)

# Get filenames and class labels from the generator
filenames = data_generator.filenames
class_labels = data_generator.classes

# Create a DataFrame to store results
embedding_df = pd.DataFrame({
    'filename': filenames,
    'class_label': class_labels,
    'embedding': [embedding.flatten() for embedding in image_embeddings]
})

# Now 'embedding_df' contains filenames, class labels, and corresponding embeddings
# You can save this DataFrame to a CSV file or use it for further analysis

625/625 [==============================] - 340s 543ms/step


In [102]:
embedding_df = embedding_df.rename(columns = {"filename": "id"})
embedding_df

,id,class_label,embedding
0,hate/1023944988365709312.jpg,0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,hate/1023946815727693824.jpg,0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,hate/1023953151907442689.jpg,0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,hate/1023958581824167937.jpg,0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,hate/1023968818794774534.jpg,0,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
...,...,...,...
19995,notHate/1117662387128209408.jpg,1,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
19996,notHate/1117662388155604995.jpg,1,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
19997,notHate/1117681682495299586.jpg,1,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
19998,notHate/1117682642688839681.jpg,1,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [103]:
embedding_df = embedding_df.drop(['class_label'], axis = 1)
embedding_df

,id,embedding
0,hate/1023944988365709312.jpg,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,hate/1023946815727693824.jpg,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,hate/1023953151907442689.jpg,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,hate/1023958581824167937.jpg,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,hate/1023968818794774534.jpg,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
...,...,...
19995,notHate/1117662387128209408.jpg,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
19996,notHate/1117662388155604995.jpg,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
19997,notHate/1117681682495299586.jpg,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
19998,notHate/1117682642688839681.jpg,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [104]:
id_embeddings = []
for id_ in embedding_df['id']:
    id_embeddings.append(id_.split("/")[-1][:-4])
embedding_df['id'] = id_embeddings
embedding_df

,id,embedding
0,1023944988365709312,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,1023946815727693824,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,1023953151907442689,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,1023958581824167937,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,1023968818794774534,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
...,...,...
19995,1117662387128209408,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
19996,1117662388155604995,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
19997,1117681682495299586,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
19998,1117682642688839681,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [105]:
text_train_embedding = pd.read_csv("train_emb_.csv")

In [106]:
ids = []
for id_ in text_train_embedding['id']:
    ids.append(str(id_))

In [107]:
text_train_embedding['id'] = ids

In [108]:
combined_train = pd.merge(text_train_embedding, embedding_df, on='id', how='inner')

In [109]:
combined_train

,id,label,0,1,2,3,4,5,6,7,...,759,760,761,762,763,764,765,766,767,embedding
0,1115290475811037185,0,-0.307445,-0.595667,2.029890,0.753241,0.239922,-0.823555,-0.451103,1.278356,...,-1.489078,0.171520,0.411397,-0.097779,-0.427549,-0.460753,0.098306,0.288707,0.114710,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,1108180634730745857,0,0.052831,0.564166,1.046895,0.224756,-0.409846,-0.277497,1.062460,0.226270,...,0.288764,-0.776590,-2.353325,-0.377366,0.052971,-0.019297,-0.267175,-0.090468,0.637216,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,1064098809314230272,0,0.453821,0.624932,1.438571,0.013055,-0.580556,-0.294235,1.263946,0.340312,...,0.034126,-0.476866,-1.265551,-0.367156,0.109852,-0.162941,-1.037338,0.258336,-0.073453,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,1114301234767368192,0,0.282132,0.913008,1.366072,0.090577,-0.450846,0.332105,0.733382,0.303934,...,0.126254,-0.418763,-0.806608,-0.303827,-0.017703,-0.029536,-0.738678,-0.086429,0.163116,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,1063542123855835136,0,-0.874735,0.816800,1.603405,-0.037106,0.050631,0.156107,-0.364708,0.777239,...,-0.788999,-0.021952,-1.685876,0.308716,-0.473874,-0.241557,0.680859,0.342039,1.229555,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,1050615572893904897,1,-0.437994,0.613172,0.511625,0.804926,0.158635,0.705959,1.646983,0.372472,...,0.792618,-0.282153,-1.619510,-0.411954,-0.312894,-0.064430,0.149199,-0.008276,0.208441,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
19996,1061275465774743554,1,0.036791,1.041342,0.405002,0.182684,0.648684,0.080356,0.834678,0.506983,...,-0.224036,-0.299380,-1.074947,0.404856,-0.377377,-0.219104,-0.542965,-0.316790,0.348185,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
19997,1056051809834414080,1,-0.308794,0.654264,0.320380,0.002315,0.166807,0.947161,0.298108,1.397049,...,-0.804429,0.150910,-0.625485,0.060437,-0.176145,-0.046389,0.646696,0.444127,-0.338408,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
19998,1024033839130148865,1,0.152254,0.383288,1.097410,0.578050,-0.545820,0.546212,1.149420,0.439744,...,-0.030085,-0.529389,-2.326682,-0.723426,0.454217,-0.403034,-0.200266,-0.322182,0.189268,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [110]:
X_train = combined_train.drop(["label"], axis = 1)

In [111]:
y_train = combined_train["label"]

In [112]:
X_train_df = pd.concat([X_train, pd.DataFrame(X_train['embedding'].apply(pd.Series))], axis=1).drop(columns=['embedding'])

In [113]:
text_train_embedding['id'][0]

'1115290475811037185'

In [114]:
embedding_df['id'][0]

'1023944988365709312'

In [115]:
combined_train

,id,label,0,1,2,3,4,5,6,7,...,759,760,761,762,763,764,765,766,767,embedding
0,1115290475811037185,0,-0.307445,-0.595667,2.029890,0.753241,0.239922,-0.823555,-0.451103,1.278356,...,-1.489078,0.171520,0.411397,-0.097779,-0.427549,-0.460753,0.098306,0.288707,0.114710,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,1108180634730745857,0,0.052831,0.564166,1.046895,0.224756,-0.409846,-0.277497,1.062460,0.226270,...,0.288764,-0.776590,-2.353325,-0.377366,0.052971,-0.019297,-0.267175,-0.090468,0.637216,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,1064098809314230272,0,0.453821,0.624932,1.438571,0.013055,-0.580556,-0.294235,1.263946,0.340312,...,0.034126,-0.476866,-1.265551,-0.367156,0.109852,-0.162941,-1.037338,0.258336,-0.073453,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,1114301234767368192,0,0.282132,0.913008,1.366072,0.090577,-0.450846,0.332105,0.733382,0.303934,...,0.126254,-0.418763,-0.806608,-0.303827,-0.017703,-0.029536,-0.738678,-0.086429,0.163116,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,1063542123855835136,0,-0.874735,0.816800,1.603405,-0.037106,0.050631,0.156107,-0.364708,0.777239,...,-0.788999,-0.021952,-1.685876,0.308716,-0.473874,-0.241557,0.680859,0.342039,1.229555,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,1050615572893904897,1,-0.437994,0.613172,0.511625,0.804926,0.158635,0.705959,1.646983,0.372472,...,0.792618,-0.282153,-1.619510,-0.411954,-0.312894,-0.064430,0.149199,-0.008276,0.208441,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
19996,1061275465774743554,1,0.036791,1.041342,0.405002,0.182684,0.648684,0.080356,0.834678,0.506983,...,-0.224036,-0.299380,-1.074947,0.404856,-0.377377,-0.219104,-0.542965,-0.316790,0.348185,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
19997,1056051809834414080,1,-0.308794,0.654264,0.320380,0.002315,0.166807,0.947161,0.298108,1.397049,...,-0.804429,0.150910,-0.625485,0.060437,-0.176145,-0.046389,0.646696,0.444127,-0.338408,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
19998,1024033839130148865,1,0.152254,0.383288,1.097410,0.578050,-0.545820,0.546212,1.149420,0.439744,...,-0.030085,-0.529389,-2.326682,-0.723426,0.454217,-0.403034,-0.200266,-0.322182,0.189268,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [67]:
text_train_embedding['id'] = text_train_embedding['id'].str.strip()
embedding_df['id'] = embedding_df['id'].str.strip()

In [117]:
X_train_df

,id,0,1,2,3,4,5,6,7,8,...,62710,62711,62712,62713,62714,62715,62716,62717,62718,62719
0,1115290475811037185,-0.307445,-0.595667,2.029890,0.753241,0.239922,-0.823555,-0.451103,1.278356,-0.293971,...,0.0,0.0,0.943271,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1108180634730745857,0.052831,0.564166,1.046895,0.224756,-0.409846,-0.277497,1.062460,0.226270,0.475940,...,0.0,0.0,0.846853,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1064098809314230272,0.453821,0.624932,1.438571,0.013055,-0.580556,-0.294235,1.263946,0.340312,-0.117352,...,0.0,0.0,0.976583,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1114301234767368192,0.282132,0.913008,1.366072,0.090577,-0.450846,0.332105,0.733382,0.303934,-0.260312,...,0.0,0.0,0.977274,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1063542123855835136,-0.874735,0.816800,1.603405,-0.037106,0.050631,0.156107,-0.364708,0.777239,0.088462,...,0.0,0.0,1.021115,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,1050615572893904897,-0.437994,0.613172,0.511625,0.804926,0.158635,0.705959,1.646983,0.372472,-0.196658,...,0.0,0.0,0.870782,0.0,0.0,0.0,0.0,0.0,0.0,0.0
19996,1061275465774743554,0.036791,1.041342,0.405002,0.182684,0.648684,0.080356,0.834678,0.506983,0.630557,...,0.0,0.0,0.442140,0.0,0.0,0.0,0.0,0.0,0.0,0.0
19997,1056051809834414080,-0.308794,0.654264,0.320380,0.002315,0.166807,0.947161,0.298108,1.397049,0.126198,...,0.0,0.0,1.098001,0.0,0.0,0.0,0.0,0.0,0.0,0.0
19998,1024033839130148865,0.152254,0.383288,1.097410,0.578050,-0.545820,0.546212,1.149420,0.439744,0.628550,...,0.0,0.0,0.918595,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [118]:
print(text_train_embedding['id'].dtype)
print(embedding_df['id'].dtype)

object
object


In [119]:
text_train_embedding['id'] = text_train_embedding['id'].str.lower()
embedding_df['id'] = embedding_df['id'].str.lower()

In [ ]:
X_train_df.to_csv("X_train_20000.csv")

In [ ]:
y_train.to_csv("y_train_20000.csv")